In [ ]:
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex AI Model Garden - Palmyra Med

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/writerai/writer-model-garden/blob/main/notebooks/pytorch_palmyra_med.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/writerai/writer-model-garden/tree/main/notebooks/pytorch_palmyra_med.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/notebooks/deploy-notebook?download_url=https://raw.githubusercontent.com/writerai/writer-model-garden/main/notebooks/pytorch_palmyra_med.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
Open in Vertex AI Workbench
    </a>
    (a Python-3 GPU notebook is required)
  </td>
</table>

## Overview

This notebook demonstrates deploying the pre-trained [Palmyra-Med-20B](https://huggingface.co/Writer/palmyra-med-20b) model on Vertex AI for online prediction.

Palmyra-Med-20B is a 20 billion parameter Large Language Model that has been uptrained on Palmyra-Large with a specialized custom curated medical dataset. 
The main objective of this model is to enhance performance in tasks related to medical dialogue and question-answering.

### Objective

- Upload the model to [Model Registry](https://cloud.google.com/vertex-ai/docs/model-registry/introduction).
- Deploy the model on [Endpoint](https://cloud.google.com/vertex-ai/docs/predictions/using-private-endpoints).
- Run online predictions for text generation.

### Costs

This tutorial uses billable components of Google Cloud:

* Vertex AI
* Cloud Storage

Learn about [Vertex AI pricing](https://cloud.google.com/vertex-ai/pricing) and [Cloud Storage pricing](https://cloud.google.com/storage/pricing), and use the [Pricing Calculator](https://cloud.google.com/products/calculator/) to generate a cost estimate based on your projected usage.

## Setup environment

**NOTE**: Jupyter runs lines prefixed with `!` as shell commands, and it interpolates Python variables prefixed with `$` into these commands.

### Colab only

Run the following commands for Colab and skip this section if you are using Workbench.

In [ ]:
import sys

if "google.colab" in sys.modules:
    ! pip3 install --upgrade google-cloud-aiplatform
    from google.colab import auth as google_auth

    google_auth.authenticate_user()

    # Restart the notebook kernel after installs.
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

### Setup Google Cloud project

1. [Select or create a Google Cloud project](https://console.cloud.google.com/cloud-resource-manager). When you first create an account, you get a $300 free credit towards your compute/storage costs.

1. [Make sure that billing is enabled for your project](https://cloud.google.com/billing/docs/how-to/modify-project).

1. [Enable the Vertex AI API and Compute Engine API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com,compute_component).

1. [Create a Cloud Storage bucket](https://cloud.google.com/storage/docs/creating-buckets) for storing experiment outputs.

Fill following variables for experiments environment:

In [ ]:
# Cloud project id.
PROJECT_ID = ""  # @param {type:"string"}

# The region you want to launch jobs in.
REGION = "us-central1"  # @param {type:"string"}

# The Cloud Storage bucket for storing experiments output. Fill it without the 'gs://' prefix.
GCS_BUCKET = ""  # @param {type:"string"}

Initialize Vertex AI API:

In [ ]:
from google.cloud import aiplatform

aiplatform.init(project=PROJECT_ID, location=REGION, staging_bucket=GCS_BUCKET)

### Define constants

In [ ]:
# The pre-built serving docker image.
# The model artifacts are embedded within the container, except for model weights which will be downloaded during deployment.
SERVE_DOCKER_URI = "us-docker.pkg.dev/vertex-ai/vertex-vision-model-garden-dockers/writer-serve"

### Define common functions

In [ ]:
def deploy_model(
    model_id: str,
    machine_type: str = "n1-standard-96",
    accelerator_type: str = "NVIDIA_TESLA_P4",
    accelerator_count: int = 4,
    model_name: str = "palmyra-med"):
    """Uploads and deploys the model to Vertex AI endpoint for prediction."""
    endpoint = aiplatform.Endpoint.create(display_name=f"{model_name}-endpoint")
    serving_env = {
        "MODEL_ID": model_id,
    }
    # If the model_id is a GCS path, use artifact_uri to pass it to serving docker.
    artifact_uri = model_id if model_id.startswith("gs://") else None
    model = aiplatform.Model.upload(
        display_name=model_name,
        serving_container_image_uri=SERVE_DOCKER_URI,
        serving_container_ports=[7080],
        serving_container_predict_route="/generate",
        serving_container_health_route="/health",
        serving_container_environment_variables=serving_env,
        artifact_uri=artifact_uri,
    )
    model.deploy(
        endpoint=endpoint,
        machine_type=machine_type,
        accelerator_type=accelerator_type,
        accelerator_count=accelerator_count,
        deploy_request_timeout=2400,
    )
    return model, endpoint

## Upload and deploy models

This section uploads the pre-trained model to Model Registry and deploys it on the Endpoint using 4 P4 GPUs. You can try different hardware configurations by changing the `machine_type`, `accelerator_type`, `accelerator count` and passing them to the `deploy_model` function.

The model deployment step will take ~30 minutes to complete.

In [ ]:
# Default configuration for this model is to use A100 (40G) to deploy Palmyra Med
# machine_type = "n1-standard-96"
# accelerator_type = "NVIDIA_TESLA_P4"
# accelerator_count = 4

model, endpoint = deploy_model(model_id="Writer/palmyra-med-20b")

## Send prediction request

In [ ]:
input_text = "Can you explain in simple terms how vaccines help our body fight diseases?"

instances = [
    {"text": input_text},
]
preds = endpoint.predict(instances=instances).predictions
for p in preds:
    print(p["generated_text"])

In [ ]:
# You can also try other parameters, like setting the temperature, max new tokens, top k, and others
# This is an example
input_text = "Can you explain in simple terms how vaccines help our body fight diseases?"

instances = [
    {"text": input_text},
]
parameters = {
    "max_new_tokens": 200,
    "temperature": 0.8
}
preds = endpoint.predict(instances=instances, parameters=parameters).predictions
for p in preds:
    print(p["generated_text"])

## Clean up

In [ ]:
# Undeploy model and delete endpoint.
endpoint.delete(force=True)

# Delete models.
model.delete()